# Stance Classification for SemEval2016
- read in the data and pre-process
- set up an LLM
- Investigate different prompting schemes
    - just context
    - context + few-shot learning
    - content + few-shot learning + reasoning

In [1]:
# Package installations to work on WIRE

! pip install transformers
! pip install langchain
! pip install accelerate
! pip install einops
! pip install deepspeed

  Using cached transformers-4.33.1-py3-none-any.whl (7.6 MB)
  Using cached huggingface_hub-0.17.1-py3-none-any.whl (294 kB)
  Using cached regex-2023.8.8-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (771 kB)
  Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Using cached safetensors-0.3.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip
  Using cached accelerate-0.22.0-py3-none-any.whl (251 kB)

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip
  Using cached einops-0.6.1-py3-none-any.whl (42 kB)

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python 

In [1]:
import os, re, pandas as pd, numpy as np, ast, json
from pprint import pprint
from tqdm import tqdm  

import torch
from langchain import PromptTemplate, FewShotPromptTemplate, HuggingFacePipeline, LLMChain
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import deepspeed

from sklearn.metrics import classification_report

from matplotlib import pyplot as plt
import seaborn as sns

2023-09-14 14:02:47.782884: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-14 14:02:48.310328: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[2023-09-14 14:02:49,308] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


# 1. Import and Preprocess Data

In [2]:
file = os.path.join("semeval","data_merged.csv")

In [3]:
df = pd.read_csv(file)

In [4]:
df.head()

,tweet_id,event,full_text,stance
0,101,Atheism,dear lord thank u for all of ur blessings forg...,disagree
1,102,Atheism,"Blessed are the peacemakers, for they shall be...",disagree
2,103,Atheism,I am not conformed to this world. I am transfo...,disagree
3,104,Atheism,Salah should be prayed with #focus and #unders...,disagree
4,105,Atheism,And stay in your houses and do not display you...,disagree


In [5]:
df.shape

(2814, 4)

# 2. Connect to LLM
- for standing up a smaller huggingface model
```python
llm = HuggingFacePipeline.from_model_id(model_id="declare-lab/flan-alpaca-gpt4-xl", task = 'text2text-generation', device=0,
                                      model_kwargs={"max_length":500, "do_sample":False})
```
- for a mid-sized, more modern, huggingface model. You can use accelerate and chance ``` device = "auto"``` to use multiple GPUs
```python
model = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map=0,
    max_length=200,
    do_sample=False,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)

llm = HuggingFacePipeline(pipeline=pipe)
```


In [6]:
# read in personal access token

with open('personal_hugginface_token.txt', 'r') as file:  
    token = file.read().strip()  

In [7]:
model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model, token=token)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    trust_remote_code=True,
    token = token,
    device_map=0,
    max_length=1024,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    repetition_penalty=2.0
)

llm = HuggingFacePipeline(pipeline=pipe)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


look at a few examples

In [8]:
question = '''What is the stance of the following social media post given in quotes toward the U.S. Army? Give the stance as either for, against, or neutral. Only return the stance and no other text.
"@vondeveen If the Army wants to actually recruit people, maybe stop breaking people and actually prosecute sexual assualt #nomorewar."
'''
llm(question)

'Stances: For'

In [9]:
question = '''What is the stance of the following social media post given in quotes toward the U.S. Army? Give the stance as either for, against, or neutral. Only return the stance and no other text.
"@artfulask I have never seen a pink-eared duck before. #Army"
'''
llm(question)

'For'

In [10]:
question = '''What is the stance of the following social media post given in quotes toward the U.S. Army? Give the stance as either for, against, or neutral. Only return the stance and no other text.
"I think the @Army helped me become disciplined. I would have surely flunked out of college chasing tail if I didn't get some discipline there. #SFL"
'''
llm(question)

'For'

# 3. Prompt Engineering for Stance Classification

## 3(a). Task-only prompt

In [11]:
# task-only prompt

task_template = '''
Classify the statement, "{statement}" as to whether it is for, against, or neutral. Only return the classification label for the statement, and no other text.
'''

task_prompt = PromptTemplate(
    input_variables=["statement"],
    template=task_template
)

### Run an example

In [12]:
# Create a sample for all examples

random_sample_df = df.sample(100)
y_true = random_sample_df['stance'].tolist()

In [13]:
llm_chain = LLMChain(prompt=task_prompt, llm=llm)

results=[]
for row in random_sample_df.iterrows():
    results.append(llm_chain.run(event=row[1]['event'], statement=row[1]['full_text']))

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [15]:
np.unique(results, return_counts=True)

(array(['For', 'For - $Label = For$', 'For - F',
        'For example if I were asked about this tweet,"#WhyIDidntReport", my response would be "#Against".',
        'For example if I were classifying a different verse from Psalms you would have received something like this,"The wicked shall be turned into hell." (for)',
        'For example if I were given "The sky has blue color." you would respond with simply "- Blue".',
        'For: "Can’tdoesntevenmanageherhusbandandshewantsnomorepower?#NoTo HillarryInTheWhitehouse🚫❌️\u200d♀️ picstormsupport”',
        'For: $# Article V$   Against$: $- SCotusCare-$ Neutral":$- w tf-',
        'For: $# Sem ST$', 'For: $#!&@*()_$%^&amp;**!!!!!',
        "For: $#!* Yeah!!!, Abortions are illegal but that don;ty stop people from getting them., The Supreme Court has spoken - they just can’ t keep redefining what a constitutional right means every time someone disagrees with their decision, I totally agree morality isn 'T about laws its personal belie

In [16]:
# converting to given labels for comparison

y_true = [str.lower(i) for i in y_true]
y_pred = []  
  
for word in results:  
    lower_word = word.lower()  
    if 'for' in lower_word:  
        y_pred.append('agree')  
    elif 'against' in lower_word or 'denies' in lower_word:  
        y_pred.append('disagree')  
    elif 'neutral' in lower_word:  
        y_pred.append('neutral')  
    else:  
        y_pred.append('neutral')

# generating the classification report
report = classification_report(y_true, y_pred)

print(report)

              precision    recall  f1-score   support

       agree       0.26      0.93      0.40        27
    disagree       0.00      0.00      0.00        47
     neutral       0.00      0.00      0.00        26

    accuracy                           0.25       100
   macro avg       0.09      0.31      0.13       100
weighted avg       0.07      0.25      0.11       100



### Run on all Data

In [15]:
# Running across the whole dataset

results = []
for index, row in tqdm(df.iterrows()):
    results.append(llm_chain.run(event=row['event'], statement=row['full_text']))

0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
2814it [34:45,  1.35it/s]


In [ ]:
np.unique(results, return_counts=True)

In [17]:
y_pred = []  
  
for word in results:  
    lower_word = word.lower()  
    if 'for' in lower_word:  
        y_pred.append('agree')  
    elif 'against' in lower_word or 'denies' in lower_word:  
        y_pred.append('disagree')  
    elif 'neutral' in lower_word:  
        y_pred.append('neutral')  
    else:  
        y_pred.append('neutral')
        
df['task_preds'] = y_pred

In [ ]:
np.unique(df['task_preds'], return_counts=True)

In [ ]:
report = classification_report(df['stance'], df['task_preds'])

print(report)

## 3(b). Context prompt

In [17]:
# context prompt

context_template = '''
The statement, "{statement}" is a social media post expressing an opinion about an entity. Classify the statement in quotes as to whether it is for, against, or neutral toward the entity, "{event}". Only return the classification label for the statement in quotes toward the entity in quotes, and no other text.
'''

context_prompt = PromptTemplate(
    input_variables=["event","statement"],
    template=context_template
)

### Run an example

In [18]:
llm_chain = LLMChain(prompt=context_prompt, llm=llm)

results=[]
for row in random_sample_df.iterrows():
    results.append(llm_chain.run(event=row[1]['event'], statement=row[1]['full_text']))

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [19]:
np.unique(results, return_counts=True)

(array(['1."Hotler In OR than CA Right Now!"#SEM ST - FOR Climate change concern',
        '@ AlySSmithhh:There She Is Pantsuit HIlllY To HiDDe ThoSE THIGHS#HILLARYSDO NORS...',
        '@ ArkBuiler32: We May Disagreebut i Love YA Man!I Don’T Want Hard Feelings# Semst',
        '@ Temp...: For',
        '@BrandiLeeHHHh Its Called Sexual Coersion And It Is The Same As RAPE!#RAPeCULTURE!!!semst',
        '@Classification: Against',
        '@Kate write ninety-five said one third Of My Generation Is MissinG.And It Can’T Be Changed but We CAN Change The Future Pro Life Youth Sem ST." For LegelizAtioN oF ABoRtiOn',
        '@Un- Climate change IS A REAL CONERN',
        '@classification Label: Against',
        '@realDonaldTrump Before We Can Defeat Isis...We Must First Defund Planned Parenthood!#MakeAmericaGreagAgain',
        'Classification Label : For', 'Classification Label: Against',
        'Classification: (against)', 'Classification: Against',
        'Classification: FOR', 'Classi

In [20]:
# converting to given labels for comparison

y_true = [str.lower(i) for i in y_true]
y_pred = []  
  
for word in results:  
    lower_word = word.lower()  
    if 'for' in lower_word:  
        y_pred.append('agree')  
    elif 'against' in lower_word:  
        y_pred.append('disagree')  
    elif 'neutral' in lower_word:  
        y_pred.append('neutral')  
    else:  
        y_pred.append('neutral')

# generating the classification report  
report = classification_report(y_true, y_pred)  
  
print(report)  

              precision    recall  f1-score   support

       agree       0.29      0.85      0.43        27
    disagree       0.53      0.19      0.28        47
     neutral       0.50      0.08      0.13        26

    accuracy                           0.34       100
   macro avg       0.44      0.37      0.28       100
weighted avg       0.46      0.34      0.28       100



### Run on all Data

In [23]:
# Running across the whole dataset

results = []
for index, row in tqdm(df.iterrows()):
    results.append(llm_chain.run(event=row['event'], statement=row['full_text']))

0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
2814it [32:05,  1.46it/s]


In [ ]:
np.unique(results, return_counts=True)

In [25]:
y_pred = []  
  
for word in results:  
    lower_word = word.lower()  
    if 'for' in lower_word:  
        y_pred.append('agree')  
    elif 'against' in lower_word:  
        y_pred.append('disagree')  
    elif 'neutral' in lower_word:  
        y_pred.append('neutral')  
    else:  
        y_pred.append('neutral')
        
df['context_preds'] = y_pred

In [ ]:
np.unique(df['context_preds'], return_counts=True)

In [ ]:
report = classification_report(df['stance'], df['context_preds'])

print(report)

## 3(c). Context + FSP

In [21]:
# Create an example template

example_template = '''
entity: {entity}
statement: {statement}
stance: {stance}
'''

example_prompt = PromptTemplate(
    input_variables=["entity","statement", "stance"],
    template=example_template
)

In [22]:
# Give some examples

examples = [
    {'entity': "Atheism",
     'statement':"Leaving Christianity enables you to love the people you once rejected. #freethinker #Christianity #SemST",
     'stance': 'for'},
    {'entity': "Climate Change is a Real Concern",
     'statement':"@AlharbiF I'll bomb anything I can get my hands on, especially if THEY aren't christian. #graham2016 #GOP #SemST",
     'stance': 'neutral'},
    {'entity': "Feminist Movement",
     'statement':"Always a delight to see chest-drumming alpha males hiss and scuttle backwards up the wall when a feminist enters the room. #manly #SemST",
     'stance': 'for'},
    {'entity': "Hillary Clinton",
     'statement':"Would you wanna be in a long term relationship with some bitch that hides her emails, & lies to your face? Then #Dontvote #SemST",
     'stance': 'against'},
    {'entity': "Legalization of Abortion",
     'statement':"@k_yoder That lady needs help, mental illness is a serious issue. #SemST",
     'stance': 'neutral'},
]

In [23]:
prefix = """
The following statements are social media posts expressing opinions about an entities. Each statement can either be for, against, or neutral toward their associated entity.
"""

suffix = '''
Now, classify the following statement toward the following entity. Only return the stance classification of the statement toward the entity and no other text.

entity: {event}
statement: {statement}
stance:
'''

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["event", "statement"],
    example_separator="\n"
)

### Run an example

In [24]:
llm_chain = LLMChain(prompt=few_shot_prompt, llm=llm)

results=[]
for row in random_sample_df.iterrows():
    results.append(llm_chain.run(event=row[1]['event'], statement=row[1]['full_text']))

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [25]:
np.unique(results, return_counts=True)

(array(['', ' Entity\tStances', '( ) Againts (X) Neutral',
        '(Note - Please do not provide any additional information beyond what has been provided)',
        '(Note - Please provide only one answer per question)',
        '(Note - Please select one option from \'for\', "again",or \'\'neutral\'\' based solely upon thesetext)',
        "(Note - Please select one option from 'for', ''again'',or `'neutral'` based soleLY ON The given statment)",
        "(Note - Please select only one option from 'for', ''aginst''or '''neutral''' based soleatelyon thesubmitted statment)",
        '(Note - You may assume all users have basic knowledge regarding these topics)',
        '(Note - You may enter multiple values separated by commas)',
        '(Note - please select only one option from \'for\', "against",or"neutral".)',
        '(Note down only "for", "against" etc.)',
        '(Note down only \'for\', "against",or “neutral" as answer)',
        '(Note down only \'for\', "againts",or"neutr

In [26]:
# converting to given labels for comparison

y_true = [str.lower(i) for i in y_true]
y_pred = []  
  
for word in results:  
    lower_word = word.lower()  
    if 'for' in lower_word:  
        y_pred.append('agree')  
    elif 'against' in lower_word:  
        y_pred.append('disagree')  
    elif 'neutral' in lower_word:  
        y_pred.append('neutral')  
    else:  
        y_pred.append('neutral')

# generating the classification report  
report = classification_report(y_true, y_pred)  
  
print(report)  

              precision    recall  f1-score   support

       agree       0.29      0.41      0.34        27
    disagree       0.75      0.19      0.31        47
     neutral       0.28      0.54      0.37        26

    accuracy                           0.34       100
   macro avg       0.44      0.38      0.34       100
weighted avg       0.50      0.34      0.33       100



### Run on all Data

In [33]:
# Running across the whole dataset

results = []
for index, row in tqdm(df.iterrows()):
    results.append(llm_chain.run(event=row['event'], statement=row['full_text']))

0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
2814it [1:12:04,  1.54s/it]


In [ ]:
np.unique(results, return_counts=True)

In [35]:
y_pred = []  
  
for word in results:  
    lower_word = word.lower()  
    if 'for' in lower_word:  
        y_pred.append('agree')  
    elif 'against' in lower_word:  
        y_pred.append('disagree')  
    elif 'neutral' in lower_word:  
        y_pred.append('neutral')  
    else:  
        y_pred.append('neutral')
        
df['fsp_preds'] = y_pred

In [ ]:
np.unique(df['fsp_preds'], return_counts=True)

In [ ]:
report = classification_report(df['stance'], df['fsp_preds'])

print(report)

## 3(d). Context + FSP + Reasoning

In [27]:
# Create an example template

example_and_reason_template = '''
entity: {entity}
statement: {statement}
stance: {stance}
reason: {reason}
'''

example_and_reason_prompt = PromptTemplate(
    input_variables=["entity","statement", "stance", "reason"],
    template=example_and_reason_template
)

In [28]:
# Give some examples

examples = [
    {'entity': "Atheism",
     'statement':'''Leaving Christianity enables you to love the people you once rejected. #freethinker #Christianity #SemST''',
     'stance': 'for',
     'reason': "The statement is for atheism as it claims leaving a religion like christianity makes one more loving."
    },
    {'entity': "Climate Change is a Real Concern",
     'statement':'''@AlharbiF I'll bomb anything I can get my hands on, especially if THEY aren't christian. #graham2016 #GOP #SemST''',
     'stance': 'neutral',
     'reason': "The statement does not explicity reference climate change and it is not clear who the author would bomb, so it is neutral toward climate change."
    },
    {'entity': "Feminist Movement",
     'statement':'''Always a delight to see chest-drumming alpha males hiss and scuttle backwards up the wall when a feminist enters the room. #manly #SemST''',
     'stance': 'for',
     'reason': "The statement supports the feminist movement as it makes fun of alpha males, who are typically seen as being against feminism."
    },
    {'entity': "Hillary Clinton",
     'statement':'''Would you wanna be in a long term relationship with some bitch that hides her emails, & lies to your face? Then #Dontvote #SemST''',
     'stance': 'against',
     'reason': "The statement is against Hillary Clinton as it calles her a bitch and claims she does bad things like hides emails and lies."
    },
    {'entity': "Legalization of Abortion",
     'statement':'''@k_yoder That lady needs help, mental illness is a serious issue. #SemST''',
     'stance': 'neutral',
     'reason': "The statement is neutral toward legalization of abortion as it does not talk about abortion and its not clear if the mentioned lady is related to abortion at all."
    }
]

In [29]:
prefix = """
The following statements are social media posts expressing opinions about an entities. Each statement can either be for, against, or neutral toward their associated entity, and each statement includes the reason why they have that stance.
"""

suffix = '''
Now, classify the following statement toward the following entity as for, against, or neutral and give the reason why you classified it as that stance. Only return the stance classification of the statement toward the entity and the reason for that classifcation, and no other text.

entity: {event}
statement: {statement}
stance:
reason:
'''

few_shot_and_reason_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_and_reason_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["event", "statement"],
    example_separator="\n"
)

### Run an example

In [30]:
llm_chain = LLMChain(prompt=few_shot_and_reason_prompt, llm=llm)

results=[]
for row in random_sample_df.iterrows():
    results.append(llm_chain.run(event=row[1]['event'], statement=row[1]['full_text']))

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [31]:
np.unique(results)

array(['', '"',
       '"Hey First world Country White Women Bitting About GenderInequality You Are Goinf To Be Fine...#KillAllMen..." this Statemnet Is Againts Thf FminsitMovrmnt Because It Uaes Hate Speech TalkngbAbout Killing All Men And SuggestThat Whjte Womn Ar eExempt From Racla Discrimination As They arE Frrth World Countrymnd Willbe Finde Even If There iS No More Mren Left On Earth',
       '(Note - Please answer only this part)',
       '(Note : You may need t oadd additional information such s context clues from previous statments etc)',
       'I am unable toread any information from this prompt because there isnothing provided regarding thestoneentionsforclassificationor reasoning behind them',
       'It seems there isnot any information provided regarding this entit',
       'It seems there may need additional context regarding this particular topic/issue; however from what was provided here...I cannot accurately determine whether "Legalisaation Of ABORTION" has been refe

In [32]:
y_pred = []  
reasons = []  
  
for result in results:  
    split_result = result.split('reason:')  
    if len(split_result) == 2:  
        # Extract stance and reason, remove leading/trailing whitespace and convert stance to lowercase  
        stance, reason = split_result
        stance = stance.replace('stance:', '').replace(',', '').strip().lower()
        reason = reason.strip()
        reasons.append(reason)
    else:
        # model did not give a reason
        stance = split_result[0]
        stance = stance.replace('stance:', '').replace(',', '').strip().lower()
        reasons.append('')
    if 'for' in stance:  
        y_pred.append('agree')  
    elif 'against' in stance:  
        y_pred.append('disagree')  
    elif 'neutral' in stance:  
        y_pred.append('neutral')
    else:  
        y_pred.append('neutral')

y_true = [str.lower(i) for i in y_true]

# generating the classification report
report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

       agree       0.28      0.48      0.35        27
    disagree       0.00      0.00      0.00        47
     neutral       0.26      0.54      0.35        26

    accuracy                           0.27       100
   macro avg       0.18      0.34      0.24       100
weighted avg       0.14      0.27      0.19       100



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Run on all Data

In [43]:
# Running across the whole dataset

results = []
for index, row in tqdm(df.iterrows()):
    results.append(llm_chain.run(event=row['event'], statement=row['full_text']))

0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
2814it [3:45:33,  4.81s/it]


In [44]:
y_pred = []  
reasons = []  
  
for result in results:  
    split_result = result.split('reason:')  
    if len(split_result) == 2:  
        # Extract stance and reason, remove leading/trailing whitespace and convert stance to lowercase  
        stance, reason = split_result
        stance = stance.replace('stance:', '').replace(',', '').strip().lower()
        reason = reason.strip()
        reasons.append(reason)
    else:
        # model did not give a reason
        stance = split_result[0]
        stance = stance.replace('stance:', '').replace(',', '').strip().lower()
        reasons.append('')
    if 'for' in stance:  
        y_pred.append('agree')  
    elif 'against' in stance:  
        y_pred.append('disagree')  
    elif 'neutral' in stance:  
        y_pred.append('neutral')
    else:  
        y_pred.append('neutral')
        
df['fsp_reason_preds'] = y_pred
df['fsp_reason_reasons'] = reasons

In [ ]:
np.unique(df['fsp_reason_preds'], return_counts=True)

In [ ]:
report = classification_report(df['stance'], df['fsp_reason_preds'])

print(report)

In [47]:
df.to_csv("semeval_llama-2-7B_results.csv")